In [2]:
import numpy as np
import sympy as sym
import pydae.build_v2 as bldr

In [3]:
u_dict = {}
f_list = []
x_list = []
g_list = []
y_list = []
h_dict = {}
params_dict = {}

## Low Pass Filter (lpf)

In [7]:
x,u = sym.symbols('x_lpf,u_lpf', real=True)
T,K = sym.symbols('T_lpf,K_lpf', real=True)

#           ┌─────────────┐        
#    u      │      K      │    x   
#    ──────►│ ─────────── ├───────►
#           │   T s + 1   │        
#           └─────────────┘        

dx = 1/T*(K*u - x)

f_list += [dx]
x_list += [ x]

u_dict.update({str(u):0.0})
h_dict.update({str(u):u, str(x):x})
params_dict.update({str(T):0.2,str(K):1.0})


## Second order system 

In [8]:
x,v,f= sym.symbols('x_ms,v_ms,f_ms', real=True)
M,D,K = sym.symbols('M_ms,D_ms,K_ms', real=True)

#           ┌────────────────┐        
#    u      │        1       │    z   
#    ───────┤ ────────────── ├───────►
#           │  Ms² + Ds + K  │        
#           └────────────────┘        


dx = v
dv = 1/M*(f - D*v - K*x)

f_list += [dx,dv]
x_list += [ x, v]

u_dict.update({str(f):0.0})
h_dict.update({'E_k_ms':1/2*M*v_ms**2, 'E_p_ms':K*x_ms})
params_dict.update({'M':1.0,'D':0.1,'K':10})

## Lead-lag 

In [6]:
u,x= sym.symbols('u_ll,x_ll', real=True)
T_1,T_2 = sym.symbols('T_1,T_2', real=True)

#            ┌───────────┐        
#     u      │  T₁s + 1  │    z   
#     ──────►│ ───────── ├───────►
#            │  T₂s + 1  │        
#            └───────────┘        
 

dx =  (u - x)/T_2


h = (u - x)*T_1/T_2 + x

f_list += [dx]
x_list += [ x]

u_dict.update({str(u):0.0})
h_dict.update({'z_ll':h})
params_dict.update({'T_1':1.0,'T_2':1.0})

### Washout 

In [9]:
u,x= sym.symbols('u_wo,x_wo', real=True)
T = sym.symbols('T_wo', real=True)

#            ┌───────────┐        
#     u      │    T·s    │    z   
#     ──────►│ ───────── ├───────►
#            │  T·s + 1  │        
#            └───────────┘        
 

dx =  (u - x)/T

h = u - x

f_list += [dx]
x_list += [ x]

u_dict.update({str(u):0.0})
h_dict.update({'z_wo':h})
params_dict.update({'T_wo':10.0})

### Derivative (aproximate)

In [10]:
u,x= sym.symbols('u_d,x_d', real=True)
T = sym.symbols('T_d', real=True)

#            ┌───────────┐        
#     u      │     s     │    z   
#     ──────►│ ───────── ├───────►
#            │  T·s + 1  │        
#            └───────────┘        
 

dx =  (u - x)/T

h = (u - x)/T

f_list += [dx]
x_list += [ x]

u_dict.update({str(u):0.0})
h_dict.update({'z_d':h})
params_dict.update({'T_d':10.0})

### PI control over LPF

In [11]:
u,x= sym.symbols('u_d,x_d', real=True)
T = sym.symbols('T_d', real=True)

#  z*   ┌───┐       ┌───┐                             ┌────────────┐         
#  ────►│+  │       │Kp │                  ┌───┐      │     1      │    z    
#       │   ├───────┤   ├─┬───────────────►│+  ├─────►│ ─────────  ├───┬───► 
#  ┌───►│-  │       └───┘ │                │   │      │  Ts + 1    │   │     
#  │    └───┘             │           ┌───►│+  │      └────────────┘   │     
#  │                      │           │    └───┘                       │     
#  │                      │   ┌─────┐ │                                │     
#  │                      │   │  1  │ │                                │     
#  │                      └───┤ ─── ├─┘                                │     
#  │                          │ Tis │                                  │     
#  │                          └─────┘                                  │     
#  │                                                                   │     
#  └───────────────────────────────────────────────────────────────────┘     
 

dx =  (u - x)/T

h = (u - x)/T

f_list += [dx]
x_list += [ x]

u_dict.update({str(u):0.0})
h_dict.update({'z_d':h})
params_dict.update({'T_d':10.0})

In [ ]:
u,x= sym.symbols('u_d,x_d', real=True)
T = sym.symbols('T_d', real=True)

#            ┌───────────┐        
#     u      │     s     │    z   
#     ──────►│ ───────── ├───────►
#            │  T·s + 1  │        
#            └───────────┘        
 

dx =  (u - x)/T

h = (u - x)/T

f_list += [dx]
x_list += [ x]

u_dict.update({str(u):0.0})
h_dict.update({'z_d':h})
params_dict.update({'T_d':10.0})

In [2]:
params_dict = {'C_1':1.0,
               'C_2':2.0,
               'C_3':3.0,
               'C_4':4.0}


u_ini_dict = {'u_1':1.0,'u_2':2.0}  # for the initialization problem
u_run_dict = {'u_1':1.0,'u_2':2.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['x_1','x_2']    # [inductor current, PI integrator]
y_ini_list = ['y_1','y_2'] # for the initialization problem
y_run_list = ['y_1','y_2'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [3]:
u_1,u_2 = sym.symbols('u_1,u_2',real=True)
dx_1 = C_1*u_1 - C_2*x_1 
dx_2 = C_3*y_1 - C_2*x_2 + y_2*x_1 

eq_y_1 = -y_1 + u_2
eq_y_2 = -y_2 + x_1

f_list = [dx_1,dx_2]
g_list = [eq_y_1,eq_y_2]
h_dict = {'z_1':x_1,'z_2':u_1*y_1}

In [4]:
sys = {'name':f"basic_1",'uz_jacs':True,
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':h_dict
      }

sys = db.build(sys,verbose=True)

check_system (time: 0.0)
computing jacobians Fx_run,Fy_run  (time: 0.030 s)
computing jacobians Gx_run,Gy_run  (time: 0.050 s)
computing jacobians Fu_run,Gu_run  (time: 0.056 s)
Matrix([[C_1*u_1 - C_2*x_1, -C_2*x_2 + C_3*y_1 + x_1*y_2]])
Matrix([[u_1, u_2]])
Matrix([[C_1, 0], [0, 0]])
computing jacobians Fx_ini,Fy_ini  (time: 0.064 s)
computing jacobians Gx_ini,Gy_ini  (time: 0.066 s)
computing jacobians Hx_run,Hy_run,Hu_run  (time: 0.06881451606750488 s)
end system  (time: 0.073 s)
computing jac_ini (time: 0.0)
computing jac_run (time: 0.000997304916381836)
computing jac_trap (time: 0.0019953250885009766)
end of jacobians computation (time: 0.014)
writting f_ini and g_ini code (time: 0.000 s)
writting f_run and g_run code (time: 0.037 s)
writting h_run code (time: 0.037 s)
converting jac_ini to sp_jac_ini  (time: 0.037 s)
running sym2rhs for sp_jac_ini (time: 0.037 s)
converting jac_run to sp_jac_run  (time: 0.069 s)
running sym2rhs for sp_jac_run (time: 0.069 s)
converting jac_trap t

In [5]:
sys['Hy_run']

Matrix([
[  0, 0],
[u_1, 0]])

In [8]:
from sympy.matrices.sparsetools import _doktocsr
from sympy import SparseMatrix


In [9]:
sp_jac_list = _doktocsr(SparseMatrix(Fu_run))  

In [10]:
sp_jac_list

[[C_1], [0], [0, 1, 1], [2, 2]]